### Imports

In [10]:
import time
import torch
import numpy as np
from typing import List
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import gensim
from utilities import *

%run preprocessing.ipynb

### Constants

In [16]:
EMBEDDING_DIM = 300
HIDDEN_SIZE = 512
NUM_LAYERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 200
VOCAB_SIZE = len(basic_arabic_letters) + 1
LABELS_SIZE = len(DIACRITICS)
PAD = 15
CONTEXTUAL_EMBEDDING_DIM=100

TRAIN_PATH = "./dataset/train.txt"
VAL_PATH = "./dataset/val.txt"
TEST_PATH = "./dataset/test.txt"
LSTM_PATH="./models/lstm.pth"
RNN_PATH="./models/rnn.pth"
CNN_PATH = "./models/cnn.pth"
CRF_PATH = "./models/crf.pth"

### GPU

In [3]:
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))

else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

No GPU available, using the CPU instead.


### Model

In [17]:
class CNN(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, num_classes=LABELS_SIZE, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS
                , pretrained_embedding=None, freeze_embedding=False,contextual_embedding_dim=CONTEXTUAL_EMBEDDING_DIM):
        super(CNN, self).__init__()
        
        # Embedding layer
        if pretrained_embedding is not None:
            self.embedding  = nn.Embedding.from_pretrained(pretrained_embedding, freeze=freeze_embedding)
        else:
            self.embedding =nn.Embedding(vocab_size, embedding_dim)
        
        # Convolutional Layer
        self.conv1d = nn.Conv1d(contextual_embedding_dim, 256, kernel_size=3, padding=1).double()
        
        # LSTM Layer
        self.lstm = nn.LSTM(256, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True).double()
        
        # Linear Layer
        self.linear = nn.Linear(2 * hidden_size, num_classes).float()

    def forward(self, sentences):
        embeddings = self.embedding(sentences)
        # if self.contextual_embedding is not None:
        #     embeddings = torch.cat([embeddings, self.contextual_embedding(sentences)], dim=2)  # Concatenate embeddings
        # Convolutional Layer
        conv_out = self.conv1d(embeddings.permute(0, 2, 1))
        conv_out = F.relu(conv_out)
        
        # LSTM Layer
        lstm_out, _ = self.lstm(conv_out.permute(0, 2, 1))
        
        # Linear Layer
        output = self.linear(lstm_out.float())

        return output

### Train

In [5]:
def data_loader(
    train_inputs, val_inputs, train_labels, val_labels, batch_size=BATCH_SIZE
):
    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(
        train_data, sampler=train_sampler, batch_size=batch_size
    )

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader


# Specify loss function
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD)


def train(
    path, model, optimizer, train_dataloader, val_dataloader=None, epochs=NUM_EPOCHS
):
    """Train the model"""

    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(
        f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}"
    )
    print("-" * 60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass
            output = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(output.view(-1, output.shape[-1]), b_labels.view(-1))
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                torch.save(model.state_dict(), path)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(
                f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}"
            )

    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")


def evaluate(model, val_dataloader):
    """
    After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode.
    model.eval()

    # Tracking variables
    val_loss = []
    val_accuracy = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Filter out the padding value
        labels_without_pad = (b_labels != PAD)

        # Get the output
        with torch.no_grad():
            output = model(b_input_ids)

        # Compute loss
        loss = loss_fn(output.view(-1, output.shape[-1]), b_labels.view(-1))
        val_loss.append(loss.item())

        # Get the predictions
        preds = output.argmax(dim=2)

        # Calculate the accuracy
        correct_predictions = ((preds == b_labels) & labels_without_pad).sum().item()
        actual_predictions = labels_without_pad.sum().item()
        accuracy = (correct_predictions / actual_predictions) * 100

        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy



In [6]:
t_model = gensim.models.Word2Vec.load('models/full_grams_cbow_100_twitter.mdl')
embedding_dim = t_model.vector_size


### Prepare Data

In [11]:
aravec_embeddings_train = []
corpus=  readFile(TRAIN_PATH)

x_train = []
y_train = []

for sentence in corpus[:10]:
	# Clean each sentence in the corpus
	char_list, diacritics_list = each_word_separate_words_and_diacritics(sentence.strip())
	words = [''.join(sublist) for sublist in char_list]
	windows=get_all_windows(' '.join(words), WINDOW_SIZE_BEFORE, WINDOW_SIZE_AFTER)
	for window in windows:
		embeddings = [t_model.wv[clean_str(word)] if clean_str(word) in t_model.wv else np.zeros(embedding_dim) for word in window]
		aravec_embeddings_train.append(np.mean(embeddings, axis=0))
	x_train.append(char_list)
	y_train.append(diacritics_list)

X_train_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in x_train for word in sentence]
X_train_padded = pad_sequence(X_train_padded, batch_first=True)

y_train_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_train for word in sentence]
y_train_padded = pad_sequence(y_train_padded, batch_first=True, padding_value=PAD)

In [13]:
aravec_embeddings_val_test = []

valid_corpus = readFile(VAL_PATH)

X_val = []
y_val = []

for sentence in valid_corpus[:5]:
	# Clean each sentence in the corpus
	# Get the char list for each word in the sentence and its corresponding diacritics
  char_list, diacritics_list = each_word_separate_words_and_diacritics(sentence.strip())
  words = [''.join(sublist) for sublist in char_list]
  windows=get_all_windows(' '.join(words), WINDOW_SIZE_BEFORE, WINDOW_SIZE_AFTER)
  for window in windows:
    embeddings = [t_model.wv[clean_str(word)] if clean_str(word) in t_model.wv else np.zeros(embedding_dim) for word in window]
    aravec_embeddings_val_test.append(np.mean(embeddings, axis=0))
  X_val.append(char_list)
  y_val.append(diacritics_list)

X_val_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in X_val for word in sentence ]
X_val_padded = pad_sequence(X_val_padded, batch_first=True)

y_val_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_val for word in sentence ]
y_val_padded = pad_sequence(y_val_padded, batch_first=True, padding_value=PAD)


### Initialize Model

In [14]:
def init_model(learning_rate=LEARNING_RATE):
    model = CNN(pretrained_embedding=torch.tensor(aravec_embeddings_train+aravec_embeddings_val_test), freeze_embedding=True)
    path = CNN_PATH

    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    return path, model, optimizer

### Execute

In [18]:
path, model, optimizer = init_model()
train_dataloader, val_dataloader = data_loader(X_train_padded, X_val_padded, y_train_padded, y_val_padded)
train(path, model, optimizer, train_dataloader, val_dataloader)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   2.392407   |  1.789307  |   36.75   |   4.00   
   2    |   1.820434   |  1.718486  |   39.54   |   3.54   
   3    |   1.774410   |  1.743381  |   40.88   |   3.69   
   4    |   1.790493   |  1.677949  |   39.67   |   3.52   
   5    |   1.685419   |  1.665183  |   39.99   |   3.47   
   6    |   1.669291   |  1.558019  |   54.49   |   3.53   
   7    |   1.582140   |  1.498251  |   48.14   |   3.45   
   8    |   1.505941   |  1.342557  |   59.31   |   3.53   
   9    |   1.351715   |  1.272343  |   60.73   |   3.52   
  10    |   1.252966   |  1.217787  |   52.68   |   3.48   


Training complete! Best accuracy: 60.73%.
